In [ ]:
# K-Means Clustering

import torch
from torchvision.ops import masks_to_boxes,remove_small_boxes,box_convert,box_iou
from .tools.utils import remove_big_boxes
from torch.nn.functional import one_hot
from tqdm import tqdm
import numpy as np

#num_anchors=15
labels = torch.from_numpy(np.load('../data/CoNIC_Challenge/labels.npy')[..., 0].astype(np.uint8)).long()
# 得到边界框
boxes_all = torch.zeros(0, 4)
bar = tqdm(labels)
for label in bar:
    if label.max() == 0:
        continue
    boxes = masks_to_boxes(one_hot(label, num_classes=label.max() + 1)[..., 1:].permute(2, 0, 1).contiguous())
    #扩大范围
    boxes[:,2:]+=2
    boxes[:,:2]-=2

    indexes = remove_small_boxes(boxes, 0.5)
    boxes = boxes[indexes]
    # 去掉标注错误大边界框
    indexes = remove_big_boxes(boxes, 70.)
    boxes = boxes[indexes]

    boxes_all = torch.cat([boxes_all, boxes], dim=0)
# 都归到相同中心位置
boxes_wh = box_convert(boxes_all, 'xyxy', 'cxcywh')
boxes_wh[..., :2] = 0
refined_boxes_all = box_convert(boxes_wh, 'cxcywh', 'xyxy')

anchors_wh = {}
for num_anchors in [4,6,8,10,12,15]:
    anchors = torch.stack([torch.zeros(num_anchors), torch.zeros(num_anchors), torch.linspace(5, 40, num_anchors), torch.linspace(5, 40, num_anchors)], dim=-1)
    anchors_xyxy = box_convert(anchors, 'cxcywh', 'xyxy')
    for _ in range(50):
        # n,15
        d_ious = 1 - box_iou(refined_boxes_all, anchors_xyxy)
        # n
        indexes = torch.argmin(d_ious, dim=1)
        for _ in range(num_anchors):
            index = indexes == _
            wh = boxes_wh[index][..., 2:].mean(dim=0)
            anchors[_, 2:] = wh
    anchors_wh[num_anchors] = anchors[...,2:]  
torch.save(anchors_wh,'anchors_wh.pt')


In [ ]:
# PanNuke
import time
import os 
import torch
from torchvision.ops import masks_to_boxes,remove_small_boxes,box_convert,box_iou
from .tools.utils import remove_big_boxes
from torch.nn.functional import one_hot
from tqdm import tqdm
import numpy as np

label_list = os.listdir('../data/PANNUKE/masks')
bar = tqdm(label_list)
boxes = torch.zeros(0,4)
for label in bar:
    label = np.load(os.path.join('../data/PANNUKE/masks',label))#256,256,6
    ins_label = torch.from_numpy(label[...,:5]).permute(2,0,1).long()
    for ins in ins_label:
        masks = torch.zeros(0,256,256)
        ins = one_hot(ins,num_classes=ins.max()+1)[...,1:].permute(2,0,1)
        for _ in ins:
            if _.max()== 0:
                continue
            else:masks=torch.cat([_.unsqueeze(0),masks],dim=0)
        box = masks_to_boxes(masks)
        boxes = torch.cat([boxes,box])

boxes[:,2:]+=2
boxes[:,:2]-=2
indexes =remove_small_boxes(boxes,1.0)
boxes = boxes[indexes]
remove_big_boxes(boxes,250.0)
boxes = boxes[indexes]
boxes = box_convert(boxes,'xyxy','cxcywh')
boxes[:,:2] = 0

print(boxes[:,2:].max())
num_anchors = 15
anchors = torch.stack([torch.zeros(num_anchors), torch.zeros(num_anchors), torch.linspace(5, 80, num_anchors), torch.linspace(5, 80, num_anchors)], dim=-1)
anchors_xyxy = box_convert(anchors, 'cxcywh', 'xyxy')
for _ in range(50):
    # n,15
    d_ious = 1 - box_iou(boxes, anchors_xyxy)
    indexes = torch.argmin(d_ious, dim=1)
    for _ in range(num_anchors):
        index = indexes == _
        wh = boxes[index][..., 2:].mean(dim=0)
        anchors[_, 2:] = wh
print(anchors[:, 2:])